## Dataset Source
[Diabetes Dataset](https://www.kaggle.com/datasets/akshaydattatraykhare/diabetes-dataset/data)   

This dataset is originally from the National Institute of Diabetes and Digestive and Kidney Diseases. The objective of the dataset is to diagnostically predict whether a patient has diabetes, based on certain diagnostic measurements included in the dataset. Several constraints were placed on the selection of these instances from a larger database. In particular, all patients here are females at least 21 years old of Pima Indian heritage.2 From the data set in the (.csv) File We can find several variables, some of them are independent (several medical predictor variables) and only one target dependent variable (Outcome).   

#### About this file
Information about dataset attributes -

Pregnancies: To express the Number of pregnancies

Glucose: To express the Glucose level in blood

BloodPressure: To express the Blood pressure measurement

SkinThickness: To express the thickness of the skin

Insulin: To express the Insulin level in blood

BMI: To express the Body mass index

DiabetesPedigreeFunction: To express the Diabetes percentage

Age: To express the age

Outcome: To express the final result 1 is Yes and 0 is No

In [2]:
import pandas as pd

### Prelimnary Data Analysis

In [4]:
df = pd.read_csv(f"diabetes.csv")

In [5]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [6]:
df.shape

(768, 9)

In [7]:
df.dtypes

Pregnancies                   int64
Glucose                       int64
BloodPressure                 int64
SkinThickness                 int64
Insulin                       int64
BMI                         float64
DiabetesPedigreeFunction    float64
Age                           int64
Outcome                       int64
dtype: object

In [8]:
df.columns #returns an Index object with the names of all columns in the df

Index(['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin',
       'BMI', 'DiabetesPedigreeFunction', 'Age', 'Outcome'],
      dtype='object')

In [9]:
df_columns = df.columns.tolist() # converts df.columns to a usable list
features = df_columns
features.remove("Outcome")
print(features)

['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI', 'DiabetesPedigreeFunction', 'Age']


In [10]:
df.isnull().sum() # Missing Values per Column

df.isnull().sum().sum() # Total Missing Values

0

In [11]:
df.duplicated().sum() # number of duplicate rows

0

In [12]:
df.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [13]:
for column in features:
    column_range = df[column].max() - df[column].min()
    print(f"{column}: range - {column_range}, min - {df[column].min()}, max - {df[column].max()}")

Pregnancies: range - 17, min - 0, max - 17
Glucose: range - 199, min - 0, max - 199
BloodPressure: range - 122, min - 0, max - 122
SkinThickness: range - 99, min - 0, max - 99
Insulin: range - 846, min - 0, max - 846
BMI: range - 67.1, min - 0.0, max - 67.1
DiabetesPedigreeFunction: range - 2.342, min - 0.078, max - 2.42
Age: range - 60, min - 21, max - 81


In [14]:
values = df['Insulin']

percentile_25 = values.quantile(0.25) # calculates the 25th percentile
percentile_75 = values.quantile(0.75) # calculates the 75th percentile

iqr = percentile_75 - percentile_25

# calculate the upper and lower thresholds
upper = percentile_75 + 1.5 * iqr
lower = percentile_25 - 1.5 * iqr # 1.5 is a value chosen out of convention

In [15]:
outlier_count = 0
outlier_df = pd.DataFrame(columns = df_columns)


for col in features:
    col_values = df[col]
    percentile_25 = col_values.quantile(0.25) # calcs 25th Percentile
    percentile_75 = col_values.quantile(0.75) # calcs 75th Percentile

    iqr = percentile_75 - percentile_25

    # calculate the upper and lower thresholds
    upper = percentile_75 + 1.5 * iqr
    lower = percentile_25 - 1.5 * iqr # 1.5 is chosen out of convention

    # identify outliers
    lower_outliers = df[col_values < lower]
    upper_outliers = df[col_values > upper]

    print(f"{col}: Upper Threshold ({round(upper,5)}), Lower Threshold ({round(lower,5)})")
    
    print(f"Is Lower Outliers Empty: {lower_outliers.empty}")
    print(f"Is Upper Outliers Empty: {upper_outliers.empty} \n\n")

    frames = [outlier_df,lower_outliers,upper_outliers]
    outlier_df = pd.concat(frames)

#     # display the outliers if present
#     for row in lower_outliers.itertuples():
#         print(row)
#         outlier_count += 1

#     for row in upper_outliers.itertuples():
#         print(row)
#         outlier_count += 1

# print(outlier_count)

Pregnancies: Upper Threshold (13.5), Lower Threshold (-6.5)
Is Lower Outliers Empty: True
Is Upper Outliers Empty: False 


Glucose: Upper Threshold (202.125), Lower Threshold (37.125)
Is Lower Outliers Empty: False
Is Upper Outliers Empty: True 


BloodPressure: Upper Threshold (107.0), Lower Threshold (35.0)
Is Lower Outliers Empty: False
Is Upper Outliers Empty: False 


SkinThickness: Upper Threshold (80.0), Lower Threshold (-48.0)
Is Lower Outliers Empty: True
Is Upper Outliers Empty: False 


Insulin: Upper Threshold (318.125), Lower Threshold (-190.875)
Is Lower Outliers Empty: True
Is Upper Outliers Empty: False 


BMI: Upper Threshold (50.55), Lower Threshold (13.35)
Is Lower Outliers Empty: False
Is Upper Outliers Empty: False 


DiabetesPedigreeFunction: Upper Threshold (1.2), Lower Threshold (-0.33)
Is Lower Outliers Empty: True
Is Upper Outliers Empty: False 


Age: Upper Threshold (66.5), Lower Threshold (-1.5)
Is Lower Outliers Empty: True
Is Upper Outliers Empty: False 

C:\Users\Superuser\AppData\Local\Temp\ipykernel_12412\1208535058.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  outlier_df = pd.concat(frames)


There are 146 Outliers in the dataset. I need to check if any of the outliers are in the same row. Currently, I think leaving the outliers in the dataset may be useful it it shows that the outliers are more likely to have diabetes. 

There are 17 rows that occur more than once in the outliers

In [18]:
display(outlier_df)

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
88,15,136,70,32,110,37.1,0.153,43,1.0
159,17,163,72,41,114,40.9,0.817,47,1.0
298,14,100,78,25,184,36.6,0.412,46,1.0
455,14,175,62,30,0,33.6,0.212,38,1.0
75,1,0,48,20,0,24.7,0.140,22,0.0
...,...,...,...,...,...,...,...,...,...
489,8,194,80,0,0,26.1,0.551,67,0.0
537,0,57,60,0,0,21.7,0.735,67,0.0
666,4,145,82,18,0,32.5,0.235,70,1.0
674,8,91,82,0,0,35.6,0.587,68,0.0


In [19]:
print(outlier_df['Outcome'].value_counts())

print(outlier_df.duplicated().sum())

Outcome
1.0    74
0.0    72
Name: count, dtype: int64
17


In [20]:
# outlier_df =  outlier_df.drop_duplicates(keep="first")

In [21]:
# print(outlier_df.duplicated().sum())

In [22]:
# print(outlier_df['Outcome'].value_counts())

In [23]:
preg_outliers = outlier_df[outlier_df['Pregnancies'] > 13.5]

In [24]:
preg_outliers.shape

(4, 9)

In [25]:
display(preg_outliers)

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
88,15,136,70,32,110,37.1,0.153,43,1.0
159,17,163,72,41,114,40.9,0.817,47,1.0
298,14,100,78,25,184,36.6,0.412,46,1.0
455,14,175,62,30,0,33.6,0.212,38,1.0


There are only 4 pregnancy outliers, but they all have diabetes, but there are no duplicate values, so the other values are within the upper and lower thresholds of the other columns

In [27]:
glucose_outliers = outlier_df[outlier_df['Glucose'] < 37.125]

In [28]:
display(glucose_outliers)

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
75,1,0,48,20,0,24.7,0.140,22,0.0
182,1,0,74,20,23,27.7,0.299,21,0.0
342,1,0,68,35,0,32.0,0.389,22,0.0
349,5,0,80,32,0,41.0,0.346,37,1.0
502,6,0,68,41,0,39.0,0.727,41,1.0


The glucose levels are 0, so they're not outliers. They're actually missing. Since it's only 5 that are missing, I think it is best to remove them. 

It seems that several columns have missing data marked as 0. I need to find them and address them